Here is my code to find the angular separation from the Einstein and ROSAT observatories. <br>
Author: Sara Negussie <br>
Date: June 13, 2022

In [74]:
#import packages 
import numpy as np
import pandas as pd
import csv
import sys
import matplotlib.pyplot as plt
from astropy import units as u

In [75]:
ROSAT = pd.read_csv("rass_short.txt", delim_whitespace=True)  #names=["A", "B"]
Einstein = pd.read_csv("ets_short.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] * u.deg #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] * u.deg #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] * u.deg #right ascention
DEC_Einstein = Einstein.iloc[:, 1] * u.deg #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio

#del_RA = np.absolute(RA_Einstein[0]-RA_ROSAT[0])
#print("del RA = \n", del_RA)
#print(SNR_ROSAT)
#print(DEC_ROSAT)
#print(SNR_Einstein)
#print(DEC_Einstein)

I will search through all of ROSAT and EINSTEIN and see if they fit the parameters mentioned above but using a modified both methods function and save them into a text file. I will cross check the two files to see if they have something that fits the parameters.

In [106]:
def coor_data(dec1, dec2, RA1, RA2, snr_1, snr_2): #takes in numbers in degree
    theta = []
    d1 = []
    d2 = []
    ra1 = []
    ra2 = []
    snr1 = []
    snr2 = []
    offset = []
    ignored_range1  = [] #these are with SNR less than 1
    ignored_range2  = [] #these are with SNR less than 1
    poserr1 = 0.0
    poserr2 = 0.0
    size = 0
  
    for i in range(np.size(dec1)):
        j = 0
        while j < np.size(dec2):
            #print("size of dec1 =", np.size(dec1), "i =", i, "j =", j)
            del_dec = np.absolute(dec1[i]-dec2[j])
            #print("del dec = \n", del_dec)
            del_RA = np.absolute(RA1[i]-RA2[j])
            #print("del RA = \n", del_RA)
            #np.sin wants numbers in radians but if you send in numbers in degree, it will do it in degree
            if del_dec < 1 * u.deg and del_RA*np.cos(dec1[i]) < 1 * u.deg: #filter
                #LoC way
                theta_rad = np.sin(dec1[i])* np.sin(dec2[i]) + np.cos(dec1[i])*np.cos(dec2[i])*np.cos(del_RA) #returns answer in rad
                #print("theta in radians = ", theta_rad)
                theta_arcsec = np.arccos(theta_rad) * 206265 * u.arcsec / u.rad
                #print("theta in arcsecs = ", theta_arcsec)
                if theta_arcsec <= 300 * u.arcsec: 
                        if snr_1[i] >= 2 and snr_1[i] < 3:
                                poserr1 = 73 * u.arcsec
                        elif snr_1[i] >= 3 and snr_1[i] < 4:
                                poserr1 = 53 * u.arcsec
                        elif snr_1[i] >= 4 and snr_1[i] < 6:
                                poserr1 = 41 * u.arcsec
                        elif snr_1[i] >= 6:
                                poserr1 = 30 * u.arcsec
                        else:
                                ignored_range1.append(i)
                                #continue 
                                #this is for cases of snr less than 2. Is this not possible or do we ignore them?
                        
                        
                        poserr2 = snr_2[j] * u.arcsec
                        confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
                        offset.append(confid)
                        theta.append(theta_arcsec) 
                        d1.append(dec1[i])
                        d2.append(dec2[j])
                        ra1.append(RA1[i])
                        ra2.append(RA2[j])
                        snr1.append(snr_1[i])
                        snr2.append(snr_2[j])
                        size += 1
                        j += 1
                else:
                  j += 1
            else:
                #theta.append(0)
                j += 1
                #continue
                
    return theta, d1, d2, offset, ra1, ra2, snr1, snr2, size, ignored_range1, ignored_range2

#theta, dec_E, dec_R, RA_E, RA_R, size_file = coor_data(DEC_Einstein, DEC_ROSAT, RA_Einstein, RA_ROSAT)

In [101]:
theta = []
size_file = []
dec_E = []
dec_R = []
RA_E = []
RA_R = []
snr_E = []
snr_R = []
confidence = []
ign_rang1 = []
ign_rang2 = []

print("Start...")

theta, dec_E, dec_R, confidence, RA_E, RA_R, snr_E, snr_R, size_file, ign_rang1, ign_rang2 = coor_data(DEC_Einstein, DEC_ROSAT, RA_Einstein, RA_ROSAT, SNR_Einstein, SNR_ROSAT)

print("End...")
#print(theta)

Start...
End...


In [120]:
print(size_file)
#g = float("{:.2f}".format(x))

ang_sep = np.zeros(size_file)
for i in range(size_file):
    #ang_sep[i] = round(theta[i].value, 3)
    ang_sep[i] = float("{:.1f}".format(theta[i].value))

confidence_new = np.zeros(size_file)
for i in range(size_file):
    #ra_E_new[i] = round(RA_E[i].value, 1)
    confidence_new[i] = float("{:.2f}".format(confidence[i]))

dec_E_new = np.zeros(size_file)
for i in range(size_file):
    #dec_E_new[i] = round(dec_E[i].value, 1)
    dec_E_new[i] = float("{:.6f}".format(dec_E[i].value))

dec_R_new = np.zeros(size_file)
for i in range(size_file):
    #dec_R_new[i] = round(dec_R[i].value, 1)
    dec_R_new[i] = float("{:.6f}".format(dec_R[i].value))

ra_E_new = np.zeros(size_file)
for i in range(size_file):
    #ra_E_new[i] = round(RA_E[i].value, 1)
    ra_E_new[i] = float("{:.6f}".format(RA_E[i].value))

ra_R_new = np.zeros(size_file)
for i in range(size_file):
    #ra_R_new[i] = round(RA_R[i].value, 1)
    ra_R_new[i] = float("{:.6f}".format(RA_R[i].value))

snr_E_new = np.zeros(size_file)
for i in range(size_file):
    #ra_E_new[i] = round(RA_E[i].value, 1)
    snr_E_new[i] = float("{:.6f}".format(snr_E[i]))

snr_R_new = np.zeros(size_file)
for i in range(size_file):
    #ra_R_new[i] = round(RA_R[i].value, 1)
    snr_R_new[i] = float("{:.6f}".format(snr_R[i]))

print("Writing file")

with open(r'angsep.txt', 'w') as fp:
        for i in range(size_file):
            if i == 0:
                fp.write("\tTheta\t Offset\t DEC-EO\t\t DEC-ROSART\t RA-EO\t\t RA-ROSAT\n")
            else:
                fp.write("%8.1f\t%.2f\t%f\t%f\t%f\t%f\n" %((ang_sep[i]), (confidence_new[i]), (dec_E_new[i]), (dec_R_new[i]), (ra_E_new[i]), (ra_R_new[i])))
            
print("finished")


1213
Writing file
finished


In [103]:
print("Angular Sep = %f ",ang_sep[4])

Angular Sep = %f  9.4


In [112]:
numbers = [4.8, 49.723, 456.781, -72.18, 5, 13]

for number in numbers:
    print(f'{number:9.1f}')

      4.8
     49.7
    456.8
    -72.2
      5.0
     13.0
